In [1]:
import rasterio
from rasterio.mask import mask
from rasterio import Affine
import geopandas as gpd
from shapely.geometry import box
from rasterio.features import geometry_mask
import numpy as np
from shapely.geometry import mapping
from shapely.geometry import Polygon
import os
from osgeo import gdal,ogr,osr

In [7]:
raster1 = rasterio.open(r'F:\wenqu\trait_map\trait_proj\site2c_d13c_trait_project_to_aviris.tif')
raster2 = rasterio.open(r'F:/wenqu/trait_map/trait_clip/site2c_d13c/26_14.tif')
shapefile = gpd.read_file('F:/wenqu/trait_map/output.shp')
geom = shapefile.geometry

out_img, out_transform = mask(raster1, geom, crop=True)



# Update the metadata of the output image:
out_meta = raster1.meta.copy()
out_meta.update({
    "driver": "GTiff",
    "height": out_img.shape[1],
    "width": out_img.shape[2],
    "transform": out_transform
})

output = 'F:/wenqu/trait_map/trait_clip/site2c_d13c/' + '26_14_1.tif'

# Write the clipped raster to a new tif file:
with rasterio.open(output, "w", **out_meta) as dest:
    for band in range(1, raster1.count + 1):
        dest.write(out_img[band-1], indexes=band)

In [ ]:
raster1 = rasterio.open(r'F:\wenqu\trait_map\site2a_la_trait_map_project3.tif')

## use shp file to clip trait image to the exactly same extent as the aviris pixel

In [2]:
folder_path = r'F:\wenqu\Aviris\site6\site6_aviris_single_pixel_shp'
extension = ".shp"
raster1 = rasterio.open(r'F:\chm\resample\site6_b123_mean_project_to_aviris.tif')
for filename in os.listdir(folder_path):
    # Check if the current item is a file
    if filename.endswith(extension) and os.path.isfile(os.path.join(folder_path, filename)):
        #output trait name
        trait_name = os.path.splitext(filename)[0] + '.tif'
        #open shapefile 
        shapefile = gpd.read_file(os.path.join(folder_path, filename))
        geom = shapefile.geometry
        out_img, out_transform = mask(raster1, geom, crop=True)
        
        # Update the metadata of the output image:
        out_meta = raster1.meta.copy()
        out_meta.update({
            "driver": "GTiff",
            "height": out_img.shape[1],
            "width": out_img.shape[2],
            "transform": out_transform
        })
        output = 'F:/chm/resample/clip/site6_mean_' + str(trait_name)
        
        # Write the clipped raster to a new tif file:
        with rasterio.open(output, "w", **out_meta) as dest:
            for band in range(1, raster1.count + 1):
                dest.write(out_img[band-1], indexes=band)
        

In [3]:
folder_path = r'F:\wenqu\Aviris\site6\site6_aviris_single_pixel_shp'
extension = ".shp"
raster1 = rasterio.open(r'F:\chm\resample\site6_b123_std_project_to_aviris.tif')
for filename in os.listdir(folder_path):
    # Check if the current item is a file
    if filename.endswith(extension) and os.path.isfile(os.path.join(folder_path, filename)):
        #output trait name
        trait_name = os.path.splitext(filename)[0] + '.tif'
        #open shapefile 
        shapefile = gpd.read_file(os.path.join(folder_path, filename))
        geom = shapefile.geometry
        out_img, out_transform = mask(raster1, geom, crop=True)
        
        # Update the metadata of the output image:
        out_meta = raster1.meta.copy()
        out_meta.update({
            "driver": "GTiff",
            "height": out_img.shape[1],
            "width": out_img.shape[2],
            "transform": out_transform
        })
        output = 'F:/chm/resample/clip_std/site6_std_' + str(trait_name)
        
        # Write the clipped raster to a new tif file:
        with rasterio.open(output, "w", **out_meta) as dest:
            for band in range(1, raster1.count + 1):
                dest.write(out_img[band-1], indexes=band)

## this is to use the aviris pixel bounding box to clip the trait image

In [5]:
# Define the mask based on the non-zero values of raster1
# mask = data1 != 0
# print(type(mask), mask.shape)
# Define the extent of raster2 as a bounding box:
xmin, ymin, xmax, ymax = raster2.bounds
bbox = box(xmin, ymin, xmax, ymax)
print(type(bbox))

<class 'shapely.geometry.polygon.Polygon'>


In [6]:
geo = gpd.GeoDataFrame({'geometry': bbox}, index=[0])
geo = geo.set_crs(raster2.crs)
# geo = geo.to_crs(raster1.crs)
print(type(geo))

<class 'geopandas.geodataframe.GeoDataFrame'>


In [7]:
print(geo)

                                            geometry
0  POLYGON ((624296.718 7476874.861, 624296.718 7...


In [7]:
# Clip raster1 using the extent of raster2:
out_img, out_transform = mask(raster1, shapes=geo.geometry, crop=True)

# Update the metadata of the output image:
out_meta = raster1.meta.copy()
out_meta.update({
    "driver": "GTiff",
    "height": out_img.shape[1],
    "width": out_img.shape[2],
    "transform": out_transform
})

output = 'F:/wenqu/trait_map/' + '26_14_1.tif'

# Write the clipped raster to a new tif file:
with rasterio.open(output, "w", **out_meta) as dest:
    for band in range(1, raster1.count + 1):
        dest.write(out_img[band-1], indexes=band)

In [3]:
import os

folder_path = "F:/wenqu/Aviris/site2a/PLSR/aviris_single_pixel"
extension = ".tif"

for filename in os.listdir(folder_path):
    # Check if the current item is a file
    if filename.endswith(extension) and os.path.isfile(os.path.join(folder_path, filename)):
        # Do something with the file, e.g. print its name
        print(filename)
        raster1 = rasterio.open(r'F:\wenqu\trait_map\site2a_la_trait_map_project3.tif')
        
        raster2 = rasterio.open('F:/wenqu/Aviris/site2a/PLSR/aviris_single_pixel/' + filename) 
        xmin, ymin, xmax, ymax = raster2.bounds
        bbox = box(xmin, ymin, xmax, ymax)
        # Reproject the bounding box of raster2 to the coordinate system of raster1:
        geo = gpd.GeoDataFrame({'geometry': bbox}, index=[0])
        geo = geo.set_crs(raster2.crs)
#         geo = geo.to_crs(raster1.crs)
        
        # Clip raster1 using the extent of raster2:
        out_img, out_transform = mask(raster1, shapes=geo.geometry, crop=True)

        # Update the metadata of the output image:
        out_meta = raster1.meta.copy()
        out_meta.update({
            "driver": "GTiff",
            "height": out_img.shape[1],
            "width": out_img.shape[2],
            "transform": out_transform
        })
        
        output = 'F:/wenqu/trait_map/clipped_trait_map/' + filename

        # Write the clipped raster to a new tif file:
        with rasterio.open(output, "w", **out_meta) as dest:
            dest.write(out_img)


site2a_0_11.tif


RasterioIOError: Attempt to create new tiff file 'F:/wenqu/trait_map/clipped_trait_map/site2a_0_11.tif' failed: No such file or directory